## Generador de Prediccion Estandar

In [1]:
#Se limpia la memoria
rm(list=ls())
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,604524,32.3,1292120,69.1,1292120,69.1
Vcells,1086805,8.3,8388608,64.0,1632674,12.5


In [2]:
semilla <- 100019
set.seed(semilla)

In [3]:
#cargo las librerias que necesito
require("data.table")
require("rpart")
require("rpart.plot")
#Contiene funciones para evaluar performance de la clasificación
library("ROCR")

library("caret")
library("ranger")

Loading required package: data.table

Loading required package: rpart

Loading required package: rpart.plot

Loading required package: ggplot2

Loading required package: lattice



In [4]:
setwd("C:/_MCD/Labo1/code")  #Establezco el Working Directory

In [5]:
dataset  <- fread("./datasets/competencia1_2022_FE_DC2.csv")

#Completo missings
dataset[is.na(dataset)] <- 0
dataset[,'clase' := as.integer(dataset[,'clase_ternaria'] == 'BAJA+2')]
dataset[ ,  'clase_ternaria' := NULL    ] 

In [6]:
dtrain  <- dataset[ foto_mes==202101 ]  #defino donde voy a entrenar
dapply  <- dataset[ foto_mes==202103 ]  #defino donde voy a aplicar el modelo

In [7]:
formula_obj  <-  formula(paste("clase ~ ."))

modelo  <- ranger( data = dtrain,  
                   formula_obj,  
                   probability=TRUE,
                  alpha=0.800293529136106,
                   num.trees= 223, 
                   min.node.size= 559, 
                   mtry= 71,
                  max.depth=13,
                   splitrule='gini'
                 ) 

Growing trees.. Progress: 11%. Estimated remaining time: 4 minutes, 53 seconds.
Growing trees.. Progress: 25%. Estimated remaining time: 3 minutes, 27 seconds.
Growing trees.. Progress: 36%. Estimated remaining time: 3 minutes, 0 seconds.
Growing trees.. Progress: 49%. Estimated remaining time: 2 minutes, 23 seconds.
Growing trees.. Progress: 60%. Estimated remaining time: 1 minute, 51 seconds.
Growing trees.. Progress: 72%. Estimated remaining time: 1 minute, 16 seconds.
Growing trees.. Progress: 85%. Estimated remaining time: 42 seconds.
Growing trees.. Progress: 99%. Estimated remaining time: 3 seconds.


Ahora grafico el arbol, va a ser muy frondoso porque la profundidad es de 10

Ahora aplico al modelo  a los datos muevos  y genero la salida para kaggle

In [8]:
#Ganancia por TP
kTPGain               <-  78000

#Pérdida por FP
kFPGain               <-  -2000

aplico el modelo a los datos nuevos

In [9]:
#Calculo ganancia sobre dataset de pruebas
dapply$score <- predict(modelo, dapply, type = 'response')$predictions[,2]

#Selecciono los que estan por encima del umbral
dapply$Predicted <- dapply$score > 1/40

#Determino ganancia
print(paste("La Ganancia es:", 3*sum(dapply$Predicted*dapply[,clase]*kTPGain + 
                                     dapply$Predicted * (0 == dapply[,clase]) * kFPGain)))

[1] "La Ganancia es: -48384000"


prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
cada columna es el vector de probabilidades 

agrego a dapply una columna nueva que es la probabilidad de BAJA+2

In [10]:
modelo

Ranger result

Call:
 ranger(data = dtrain, formula_obj, probability = TRUE, alpha = 0.800293529136106,      num.trees = 223, min.node.size = 559, mtry = 71, max.depth = 13,      splitrule = "gini") 

Type:                             Probability estimation 
Number of trees:                  223 
Sample size:                      161342 
Number of independent variables:  269 
Mtry:                             71 
Target node size:                 559 
Variable importance mode:         none 
Splitrule:                        gini 
OOB prediction error (Brier s.):  0.004997648 

In [11]:
fwrite( dapply[ , list(numero_de_cliente, Predicted) ], #solo los campos para Kaggle
        file= "./exp/RANGER.csv", 
        sep= "," )

In [12]:
par(las=2,mar=c(5,15,4,2))
barplot(sort(importance(modelo),decreasing=TRUE)[1:15],horiz=TRUE)

ERROR: Error in importance.ranger(modelo): No variable importance found. Please use 'importance' option when growing the forest.
